#### Generate features for ML predictions of adverse outcomes and in-hospital rehabilitation needs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.ticker as mtick
from matplotlib.dates import DateFormatter
from datetime import timedelta
from datetime import datetime
from tqdm import tqdm

##### Load routine data and cohort

In [ ]:
inp_data = pd.read_csv('')
print(inp_data.shape, inp_data.ppid.nunique())

##### Set 65+ cohort

In [ ]:
print(len(inp_data[inp_data.total_count_all>0]) / len(inp_data))
print(len(inp_data[inp_data.total_count_rehab>0]) / len(inp_data))

In [ ]:
inp_data.shape

In [ ]:
## Comorbidities
smr_data = pd.read_csv('', sep='\t', low_memory=False, encoding='iso-8859-1')
gp_data = pd.read_csv('', sep='\t', low_memory=False, encoding='iso-8859-1')
## TRAK Questionnaires
trq_4at = pd.read_csv('', sep='\t', low_memory=False)
trq_BBF = pd.read_csv('', sep='\t', low_memory=False, encoding='iso-8859-1')
trq_falls = pd.read_csv('', sep='\t', low_memory=False)
trq_mobility = pd.read_csv('', sep='\t', low_memory=False)
trq_MRSA = pd.read_csv('', sep='\t', low_memory=False)
trq_MUST = pd.read_csv('', sep='\t', low_memory=False)
trq_nutr = pd.read_csv('', sep='\t', low_memory=False, encoding='iso-8859-1')
trq_RUB = pd.read_csv('', sep='\t', low_memory=False)
trq_wt = pd.read_csv('', sep='\t', low_memory=False)
## ICU/HDU data
ww_hai = pd.read_csv('', sep='\t', low_memory=False)
ww_howie = pd.read_csv('', sep='\t', low_memory=False)
ww_ph = pd.read_csv('', sep='\t', low_memory=False)
ww_sics = pd.read_csv('', sep='\t', low_memory=False)
ww_adm = pd.read_csv('', sep='\t', low_memory=False)
## Demographics
demo_data = pd.read_csv('', sep='\t', low_memory=False)

##### Select features in inpatient cohort

In [ ]:
inp_data.columns.tolist()

In [ ]:
inp_data.isnull().sum().tail(50)

In [ ]:
inp_ch = inp_data[['ppid', 'EpisodeNumber', 'AdmissionDate', 'ED_adate_dt', 'IndexAttDate', 'HOSP_adt', 'DischargeDate', 'HOSP_ddt',
                   'breq_dt', 'HOSP_FCC_dt', 'HOSP_FAS_dt',
                   'AgeAtAdmission', 'Sex', 'simd_dec', 'PresentingCondition', 'AttendedED',
                   'arrival_mode', 'triage_code',
                   'gt_m', 'gt_cc', 'gt_es_hosp', 'gt_dd',
                  'total_count_all', 'total_count_rehab']]
print(inp_ch.shape)
### Get only valid ED dates
inp_ch = inp_ch[~inp_ch.ED_adate_dt.isnull()]
print(inp_ch.shape)
inp_ch['HOSP_adt'] = pd.to_datetime(inp_ch['HOSP_adt'])
inp_ch['ED_adate_dt'] = pd.to_datetime(inp_ch['ED_adate_dt'])
inp_ch['IndexAttDate'] = pd.to_datetime(inp_ch['ED_adate_dt']).dt.date
inp_ch['breq_dt'] = pd.to_datetime(inp_ch['breq_dt'])
inp_ch['HOSP_FCC_dt'] = pd.to_datetime(inp_ch['HOSP_FCC_dt'])
inp_ch['HOSP_FAS_dt'] = pd.to_datetime(inp_ch['HOSP_FAS_dt'])
inp_ch['HOSP_FAS_dt'] = np.where(inp_ch['HOSP_FAS_dt'].isnull(), inp_ch['HOSP_FCC_dt'], inp_ch['HOSP_FAS_dt'])
#inp_ch['ED_time_mins'] = (inp_ch['HOSP_adt'] - inp_ch['ED_adate_dt']) / pd.Timedelta(minutes=1)
#inp_ch = inp_ch[inp_ch.ED_time_mins >= 0]
print(inp_ch.shape)
inp_ch['Sex_F'] = np.where(inp_ch.Sex == 'F', 1, 0)
inp_ch['simd_dec'] = inp_ch['simd_dec'].astype(int)
inp_ch = inp_ch.drop('Sex', axis=1)
inp_ch = inp_ch.sort_values(['ppid', 'EpisodeNumber', 'HOSP_adt'])
inp_ch = pd.get_dummies(inp_ch, columns=['arrival_mode', 'PresentingCondition'])
inp_ch = inp_ch.rename(columns=lambda x: x.replace(' ', '_'))
#### Set index date
inp_ch['HOSP_adt_s'] = inp_ch['HOSP_adt']
#inp_ch['HOSP_adt'] = inp_ch['ED_adate_dt']
#inp_ch['AdmissionDate'] = pd.to_datetime(inp_ch['ED_adate_dt']).dt.date

In [ ]:
### Set point of sampling
inp_ch['coh_idx'] = 1
inp_ch['HOSP_adt_s'] = inp_ch['HOSP_adt']
inp_ch['HOSP_adt'] = pd.to_datetime(inp_ch['HOSP_adt'] + timedelta(hours=72))
### Filter pts still in study
inp_ch = inp_ch[inp_ch['HOSP_adt'] < inp_ch['HOSP_ddt']]
print(inp_ch.shape)
inp_ch = pd.merge(inp_ch, demo_data[['ppid', 'DateOfDeath']], how='left', on='ppid')
inp_ch = inp_ch[(inp_ch['HOSP_adt'] < inp_ch['DateOfDeath'])|(inp_ch['DateOfDeath'].isna())]
print(inp_ch.shape)
#inp_ch['AdmissionDate'] = inp_ch['IndexAttDate']

In [ ]:
inp_ch.shape

In [ ]:
inp_ch = inp_ch.drop(['arrival_mode_M', 'arrival_mode_H', 'arrival_mode_C'], axis=1)

In [ ]:
inp_ch.columns

#### Parse WardWatcher features

In [ ]:
def get_ww_features(inp_ch, ww_adm, ww_hai, ww_howie, ww_ph, ww_sics,
                   n_cohorts=10, window=15):
    ### WW ADM
    ww_adm = ww_adm[['ppid', 'CriticalCareUniqueID', 'Unit_Admit_Date', 'Unit_Admit_Time',
                     'Unit_Disch_date', 'Unit_Disch_Time', 'ICNARC_1_Diag', 'ICNARC_4_Diag']].dropna()
    ww_adm['CC_adate'] = pd.to_datetime(ww_adm['Unit_Admit_Date'].astype(str) + ' ' + ww_adm['Unit_Admit_Time'].astype(str), errors='coerce')
    ww_adm['CC_ddate'] = pd.to_datetime(ww_adm['Unit_Disch_date'].astype(str) + ' ' + ww_adm['Unit_Disch_Time'].astype(str), errors='coerce')
    ww_adm['ICNARC_1_Diag'] = ww_adm['ICNARC_1_Diag'].astype(int)
    ww_adm['ICNARC_4_Diag'] = ww_adm['ICNARC_4_Diag'].astype(int)
    ### Previous ICU admission reason
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 100570) & (ww_adm['ICNARC_1_Diag'] <= 100780), 'gastrointestinal', 'other')
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 101670) & (ww_adm['ICNARC_1_Diag'] <= 101850), 'gastrointestinal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 301050) & (ww_adm['ICNARC_1_Diag'] <= 301489), 'gastrointestinal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 301491) & (ww_adm['ICNARC_1_Diag'] <= 301495), 'gastrointestinal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 302780) & (ww_adm['ICNARC_1_Diag'] <= 302910), 'gastrointestinal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] == 308600), 'gastrointestinal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 308180) & (ww_adm['ICNARC_1_Diag'] <= 308190), 'gastrointestinal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 100220) & (ww_adm['ICNARC_1_Diag'] <= 100390), 'respiratory', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 101500) & (ww_adm['ICNARC_1_Diag'] <= 101570), 'respiratory', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] == 102180), 'respiratory', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 300440) & (ww_adm['ICNARC_1_Diag'] <= 300799), 'respiratory', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 309010) & (ww_adm['ICNARC_1_Diag'] <= 309020), 'respiratory', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 101500) & (ww_adm['ICNARC_1_Diag'] <= 101570), 'respiratory', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 100790) & (ww_adm['ICNARC_1_Diag'] <= 100870), 'renal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 101860) & (ww_adm['ICNARC_1_Diag'] <= 101960), 'renal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] == 301490), 'renal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 301500) & (ww_adm['ICNARC_1_Diag'] <= 301645), 'renal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 308300) & (ww_adm['ICNARC_1_Diag'] <= 308340), 'renal', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 100000) & (ww_adm['ICNARC_1_Diag'] <= 100210), 'cardiovascular', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 101240) & (ww_adm['ICNARC_1_Diag'] <= 101490), 'cardiovascular', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] >= 300010) & (ww_adm['ICNARC_1_Diag'] <= 300430), 'cardiovascular', ww_adm['ww_prev_syst_code'])
    ww_adm['ww_prev_syst_code'] = np.where((ww_adm['ICNARC_1_Diag'] == 0), 'unk', ww_adm['ww_prev_syst_code'])
    ### Previous surgery code
    ww_adm['ww_prev_surg_code'] = np.where((ww_adm['ICNARC_4_Diag'] >= 302780) & (ww_adm['ICNARC_4_Diag'] <= 302910), 'gastrointestinal', 'other')
    ww_adm['ww_prev_surg_code'] = np.where((ww_adm['ICNARC_4_Diag'] == 302670), 'cardiac', ww_adm['ww_prev_surg_code'])
    ww_adm['ww_prev_surg_code'] = np.where((ww_adm['ICNARC_4_Diag'] >= 302680) & (ww_adm['ICNARC_4_Diag'] <= 302690), 'cardiac', ww_adm['ww_prev_surg_code'])
    ww_adm['ww_prev_surg_code'] = np.where((ww_adm['ICNARC_4_Diag'] >= 308110) & (ww_adm['ICNARC_4_Diag'] <= 308120), 'cardiac', ww_adm['ww_prev_surg_code'])
    ww_adm['ww_prev_surg_code'] = np.where((ww_adm['ICNARC_4_Diag'] >= 308160) & (ww_adm['ICNARC_4_Diag'] <= 308170), 'cardiac', ww_adm['ww_prev_surg_code'])
    ww_adm['ww_prev_surg_code'] = np.where((ww_adm['ICNARC_4_Diag'] == 309000), 'cardiac', ww_adm['ww_prev_surg_code'])
    ww_adm['ww_prev_surg_code'] = np.where((ww_adm['ICNARC_4_Diag'] >= 303120) & (ww_adm['ICNARC_4_Diag'] <= 303160), 'urological', ww_adm['ww_prev_surg_code'])
    ww_adm['ww_prev_surg_code'] = np.where((ww_adm['ICNARC_4_Diag'] == 0), 'unk', ww_adm['ww_prev_surg_code'])
    ww_adm = pd.get_dummies(ww_adm, columns=['ww_prev_syst_code', 'ww_prev_surg_code'])
    ww_adm = ww_adm.drop(['Unit_Admit_Date', 'Unit_Admit_Time',
                     'Unit_Disch_date', 'Unit_Disch_Time', 'ICNARC_4_Diag', 'ICNARC_1_Diag'], axis=1)
    ### WW HAI
    ww_hai = ww_hai.fillna(0)
    ww_hai['ww_antimicr'] = np.where((ww_hai['Antimicrobials_ICU_Day1or2']==1)|(ww_hai['Antimicrobials_ICUAdmit']==1), 1, 0)
    ww_hai['ww_surgery'] = np.where((ww_hai['Surgery_7Days_After']==1)|(ww_hai['Surgery_7Days_Prior']==1), 1, 0)
    ww_hai = ww_hai.merge(ww_adm[['ppid', 'CriticalCareUniqueID', 'CC_adate', 'CC_ddate']], how='left', on=['ppid', 'CriticalCareUniqueID'])
    ww_hai = ww_hai[ww_hai.CC_adate.notnull()]
    ww_hai['CC_adate'] = pd.to_datetime(ww_hai['CC_adate'])
    ww_hai = ww_hai[['ppid', 'CC_adate', 'ww_antimicr', 'ww_surgery', 'Trauma_Admission']].rename(columns={'Trauma_Admission': 'ww_trauma'})
    ww_hai[['ww_antimicr', 'ww_surgery', 'ww_trauma']] = ww_hai[['ww_antimicr', 'ww_surgery', 'ww_trauma']].astype(np.int8)
    ### WW HOWIE
    ww_howie = ww_howie.fillna(0)
    ww_howie = ww_howie[['ppid', 'CriticalCareUniqueID', 'Date', 'Resp_Manage_Ventilator', 'Resp_Manage_CPAP', 'Resp_Manage_O2_Less50pc',
    'Airway_Manage_ETT', 'Multiple_IV_VA_Drugs', 'Blood_Purify', 'ICP_Monitor', 'Neuro_OneToOne', 'CNS_Depression',
    'Nutrition_Parenteral', 'Nutrition_Enteral']].rename(columns={'Date': 'MeasureDate',
                                                                  'Resp_Manage_Ventilator': 'ww_iv_need', 
                                                                  'Resp_Manage_CPAP': 'ww_cpap_need',
                                                                  'Resp_Manage_O2_Less50pc': 'ww_02_b50pc',
                                                                  'Airway_Manage_ETT': 'ww_airway_eti',
                                                                  'Multiple_IV_VA_Drugs': 'ww_multi_iv_drugs',
                                                                  'Blood_Purify': 'ww_ac_renal_repl',
                                                                  'ICP_Monitor': 'ww_inv_neuro_monitor',
                                                                  'Neuro_OneToOne': 'ww_neuro_1to1_need',
                                                                  'CNS_Depression': 'ww_cns_depression',
                                                                  'Nutrition_Parenteral': 'ww_nutr_par_need',
                                                                  'Nutrition_Enteral': 'ww_nutr_ent_need'})
    ww_howie = ww_howie.merge(ww_adm[['ppid', 'CriticalCareUniqueID', 'CC_adate', 'CC_ddate']], how='left', on=['ppid', 'CriticalCareUniqueID'])
    ww_howie = ww_howie[ww_howie.CC_adate.notnull()]
    ww_howie = ww_howie[ww_howie.MeasureDate.notnull()]
    ww_howie['CC_adate'] = pd.to_datetime(ww_howie['CC_adate'])
    ww_howie['MeasureDate'] = pd.to_datetime(ww_howie['MeasureDate'])
    ### WW Physiology
    ww_ph = ww_ph[['ppid', 'CriticalCareUniqueID', 'AP2']]
    ww_ph = ww_ph.merge(ww_adm[['ppid', 'CriticalCareUniqueID', 'CC_adate', 'CC_ddate']], how='left', on=['ppid', 'CriticalCareUniqueID'])
    ww_ph = ww_ph[ww_ph.CC_adate.notnull()]
    ww_ph['CC_adate'] = pd.to_datetime(ww_ph['CC_adate'])
    ww_ph['AP2'] = ww_ph['AP2'].astype(int)
    ww_ph = ww_ph.rename(columns={'AP2': 'ww_AP2'})
    ### WW SICS
    ww_sics = ww_sics[['ppid', 'CriticalCareUniqueID', 'Vent_days']]
    ww_sics = ww_sics.merge(ww_adm[['ppid', 'CriticalCareUniqueID', 'CC_adate', 'CC_ddate']], how='left', on=['ppid', 'CriticalCareUniqueID'])
    ww_sics = ww_sics[ww_sics.CC_adate.notnull()]
    ww_sics['CC_adate'] = pd.to_datetime(ww_ph['CC_adate'])
    ww_sics['Vent_days'] = ww_sics['Vent_days'].astype(int)
    ww_sics = ww_sics[ww_sics['Vent_days']<100]
    ww_sics = ww_sics.rename(columns={'Vent_days': 'ww_vent_days'})
    ### Pipeline
    full_coh = pd.DataFrame()
    inp_ch['HOSP_adt'] = pd.to_datetime(inp_ch['HOSP_adt'])
    for i in range(1, n_cohorts+1, window):
        print('Processing cohort:', i)
        cur_coh = inp_ch[inp_ch['coh_idx']==i]
        #pr_start = inp_ch[inp_ch['coh_idx']==i]['HOSP_iadate']
        ww_adm_ch = pd.merge(ww_adm, cur_coh[['ppid', 'EpisodeNumber', 'HOSP_adt']], how='left', on=['ppid'])
        ww_adm_ch = ww_adm_ch[(ww_adm_ch['CC_adate'] < (ww_adm_ch['HOSP_adt'] + timedelta(hours=0)))&(ww_adm_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        ww_adm_ch = ww_adm_ch.sort_values(['ppid', 'CC_adate']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        ww_sics_ch = pd.merge(ww_sics, cur_coh[['ppid', 'EpisodeNumber', 'HOSP_adt']], how='left', on=['ppid'])
        ww_sics_ch = ww_sics_ch[(ww_sics_ch['CC_adate'] < (ww_sics_ch['HOSP_adt'] + timedelta(hours=0)))&(ww_sics_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        ww_sics_ch = ww_sics_ch.sort_values(['ppid', 'CC_adate']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        ww_ph_ch = pd.merge(ww_ph, cur_coh[['ppid', 'EpisodeNumber', 'HOSP_adt']], how='left', on=['ppid'])
        ww_ph_ch = ww_ph_ch[(ww_ph_ch['CC_adate'] < (ww_ph_ch['HOSP_adt'] + timedelta(hours=0)))&(ww_ph_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        ww_ph_ch = ww_ph_ch.sort_values(['ppid', 'EpisodeNumber', 'CC_adate']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        ww_howie_ch = pd.merge(ww_howie, cur_coh[['ppid', 'EpisodeNumber', 'HOSP_adt']], how='left', on=['ppid'])
        ww_howie_ch = ww_howie_ch[(ww_howie_ch['CC_adate'] < (ww_howie_ch['HOSP_adt'] + timedelta(hours=0)))&(ww_howie_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        ww_howie_ch = ww_howie_ch[(ww_howie_ch['MeasureDate'] < (ww_howie_ch['HOSP_adt'] + timedelta(hours=0)))]
        ww_howie_ch = ww_howie_ch.sort_values(['ppid', 'EpisodeNumber', 'CC_adate']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        ww_hai_ch = pd.merge(ww_hai, cur_coh[['ppid', 'EpisodeNumber', 'HOSP_adt']], how='left', on=['ppid'])
        ww_hai_ch = ww_hai_ch[(ww_hai_ch['CC_adate'] < (ww_hai_ch['HOSP_adt'] + timedelta(hours=0)))&(ww_hai_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        ww_hai_ch = ww_hai_ch.sort_values(['ppid', 'EpisodeNumber', 'CC_adate']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        cur_coh = pd.merge(cur_coh, ww_adm_ch.drop(['CriticalCareUniqueID', 'CC_adate', 'CC_ddate'], axis=1), how='left', on=['ppid', 'EpisodeNumber', 'HOSP_adt'])
        cur_coh = pd.merge(cur_coh, ww_howie_ch.drop(['MeasureDate', 'CriticalCareUniqueID', 'CC_adate', 'CC_ddate'], axis=1), how='left', on=['ppid', 'EpisodeNumber', 'HOSP_adt'])
        cur_coh = pd.merge(cur_coh, ww_hai_ch.drop(['CC_adate'], axis=1), how='left', on=['ppid', 'EpisodeNumber', 'HOSP_adt'])
        cur_coh = pd.merge(cur_coh, ww_ph_ch.drop(['CriticalCareUniqueID', 'CC_adate', 'CC_ddate'], axis=1), how='left', on=['ppid', 'EpisodeNumber', 'HOSP_adt'])
        cur_coh = pd.merge(cur_coh, ww_sics_ch.drop(['CriticalCareUniqueID', 'CC_adate', 'CC_ddate'], axis=1), how='left', on=['ppid', 'EpisodeNumber', 'HOSP_adt'])
        cur_coh[['ww_iv_need', 'ww_cpap_need', 'ww_02_b50pc', 'ww_airway_eti',
       'ww_multi_iv_drugs', 'ww_ac_renal_repl', 'ww_inv_neuro_monitor',
       'ww_cns_depression', 'ww_neuro_1to1_need', 'ww_nutr_par_need',
       'ww_nutr_ent_need', 'ww_antimicr', 'ww_surgery', 'ww_trauma']] = cur_coh[['ww_iv_need', 'ww_cpap_need', 'ww_02_b50pc', 'ww_airway_eti',
       'ww_multi_iv_drugs', 'ww_ac_renal_repl', 'ww_inv_neuro_monitor',
       'ww_cns_depression', 'ww_neuro_1to1_need', 'ww_nutr_par_need',
       'ww_nutr_ent_need', 'ww_antimicr', 'ww_surgery', 'ww_trauma']].round().fillna(0).astype(np.int8)
        cur_coh[[col for col in cur_coh if 'ww_prev' in col]] = cur_coh[[col for col in cur_coh if 'ww_prev' in col]].fillna(0).astype(np.int8)
        cur_coh[['ww_AP2', 'ww_vent_days']] = cur_coh[['ww_AP2', 'ww_vent_days']].round().fillna(0).astype(np.int16)
        full_coh = pd.concat([full_coh, cur_coh], axis=0)

    return full_coh

In [ ]:
inp_ch = get_ww_features(inp_ch, ww_adm, ww_hai, ww_howie, ww_ph, ww_sics)

In [ ]:
inp_ch.shape

In [ ]:
inp_ch.isnull().any().sum()

In [ ]:
for col in inp_ch.columns:
    if 'ww_' in col:
        print(inp_ch[col].value_counts())

#### Parse TrakQ features

In [ ]:
def get_trakq_features(inp_ch, trq_4at, trq_BBF, trq_falls, trq_mobility,
                   trq_MRSA, trq_MUST, trq_nutr, trq_RUB, trq_wt, n_cohorts=10, window=15):
    ### TRQ 4AT
    trq_4at = trq_4at[['ppid', 'DateOfAssessment', '4AT score']].rename(columns={'4AT score': 'trQ_4at'})
    trq_4at['DateOfAssessment'] = pd.DatetimeIndex(pd.to_datetime(trq_4at['DateOfAssessment'])).tz_localize(None)
    
    ### TRQ BBF
    cat_dict = {'Y': 1, 'N': 0}
    trq_BBF = trq_BBF[['ppid', 'DateIdentified', 'Is_the_patientâs_bladder_drained_by_a_urinary_catheter?_Code',
                      'Is_the_patient_incontinent_of_urine?_Code', 'Dysuria_(pain_passing_urine)?_Code',
                      'Frequency_(greater_than_6_times_per_day)?_Code', 'Nocturia_(greater_than_twice_per_night)?_Code',
                      'Is_the_patient_incontinent_of_faeces?_Code', 'Constipation?_Code', 'Diarrhoea?_Code',
                      'Blood_in_stools?_Code', 'Is_the_patient_on_any_current_medication_for_bowels?_Code']].fillna('N')

    for rc in ['Is_the_patientâs_bladder_drained_by_a_urinary_catheter?_Code',
                      'Is_the_patient_incontinent_of_urine?_Code', 'Dysuria_(pain_passing_urine)?_Code',
                      'Frequency_(greater_than_6_times_per_day)?_Code', 'Nocturia_(greater_than_twice_per_night)?_Code',
                      'Is_the_patient_incontinent_of_faeces?_Code', 'Constipation?_Code', 'Diarrhoea?_Code',
                      'Blood_in_stools?_Code', 'Is_the_patient_on_any_current_medication_for_bowels?_Code']:
        trq_BBF[rc] = trq_BBF[rc].map(cat_dict).astype(np.int8)
    
    trq_BBF = trq_BBF.rename(
        columns={'Is_the_patientâs_bladder_drained_by_a_urinary_catheter?_Code': 'trQ_bwm_urinary_catheterisation',
                'Is_the_patient_incontinent_of_urine?_Code': 'trQ_bwm_urinary_incontinence',
                'Dysuria_(pain_passing_urine)?_Code': 'trQ_bwm_dysuria',
                'Frequency_(greater_than_6_times_per_day)?_Code': 'trQ_bwm_>6times_per_day',
                'Nocturia_(greater_than_twice_per_night)?_Code': 'trQ_bwm_nocturia_>2_per_night',
                'Is_the_patient_incontinent_of_faeces?_Code': 'trQ_bwm_faeces_incontinence',
                'Constipation?_Code': 'trQ_bwm_constipation',
                'Diarrhoea?_Code': 'trQ_bwm_diarrhoea',
                'Blood_in_stools?_Code': 'trQ_bwm_blood_in_stools',
                'Is_the_patient_on_any_current_medication_for_bowels?_Code': 'trQ_bwm_medication'})
    trq_BBF['DateIdentified'] = pd.to_datetime(trq_BBF['DateIdentified'])
    
    ### TRQ FALLS
    trq_falls = trq_falls[['ppid', 'DateOfAssessment', 'Has_the_patient_fallen_in_the_last_6_months_code',
                      'Based_on_your_clinical_judgement_is_this_patient_at_high_risk_of_falls_code']].fillna('N')
    for rc in ['Has_the_patient_fallen_in_the_last_6_months_code',
                      'Based_on_your_clinical_judgement_is_this_patient_at_high_risk_of_falls_code']:
        trq_falls[rc] = trq_falls[rc].map(cat_dict).astype(np.int8)
    trq_falls = trq_falls.rename(
        columns={'Has_the_patient_fallen_in_the_last_6_months_code': 'trQ_falls_within_6_months',
                'Based_on_your_clinical_judgement_is_this_patient_at_high_risk_of_falls_code': 'trQ_falls_clinical_risk'})
    trq_falls['DateOfAssessment'] = pd.DatetimeIndex(pd.to_datetime(trq_falls['DateOfAssessment'])).tz_localize(None)
    
    ### TRQ MOBILITY
    trq_mobility = trq_mobility[['ppid', 'DateOfAssessment', 'Walking', 'Toileting', 'Bathing/showering',
       'Rolling in bed', 'Moving up the bed',
       'Getting out of bed', 'Getting into bed',
       'Sit to stand to sit', 'Lateral transfers',
       'Up from floor']].fillna('NA_BREST')
    trq_mobility = trq_mobility.rename(
        columns={'Walking': 'trQ_mobility_walking',
                'Toileting': 'trQ_mobility_toileting',
                'Bathing/showering': 'trQ_mobility_bathing',
                'Rolling in bed': 'trQ_mobility_bed_rolling',
                'Moving up the bed': 'trQ_mobility_bed_moveup',
                'Getting out of bed': 'trQ_mobility_bed_out',
                'Getting into bed': 'trQ_mobility_bed_in',
                'Sit to stand to sit': 'trQ_mobility_sss',
                'Lateral transfers': 'trQ_mobility_lateral',
                'Up from floor': 'trQ_mobility_floorup'})
    trq_mobility['DateOfAssessment'] = pd.DatetimeIndex(pd.to_datetime(trq_mobility['DateOfAssessment'])).tz_localize(None)
    for col in trq_mobility.columns:
        if 'trQ_mobility_' in col:
            trq_mobility[col] = trq_mobility[col].fillna('Not assessed').astype(str)
            trq_mobility[col] = np.where(trq_mobility[col].str.contains('Independent'), 'INDEPENDENT',
                                         np.where(trq_mobility[col].str.contains('Not applicable'), 
                                                       'BED_REST', 
                                                       np.where((trq_mobility[col].str.contains('Assistance'))|
                                                       (trq_mobility[col].str.contains('Zimmer / Rollator'))|
                                                       (trq_mobility[col].str.contains('Stick'))|
                                                       (trq_mobility[col].str.contains('Crutches'))| 
                                                       (trq_mobility[col].str.contains('Glidesheet'))|
                                                       (trq_mobility[col].str.contains('lifting hoist'))|
                                                       (trq_mobility[col].str.contains('Stand aid'))|
                                                        (trq_mobility[col].str.contains('Patient turning device'))|
                                                        (trq_mobility[col].str.contains('Commode'))|
                                                        (trq_mobility[col].str.contains('Bedpan'))|
                                                        (trq_mobility[col].str.contains('Bathing hoist'))|
                                                        (trq_mobility[col].str.contains('Shower chair'))|
                                                        (trq_mobility[col].str.contains('Lateral transfer board'))|
                                                        (trq_mobility[col].str.contains('Hoverjack'))|
                                                        (trq_mobility[col].str.contains('Lifting cushion')),
                                                       'ASSISTANCE', 'OTHER')))
    ### TRQ MRSA
    trq_MRSA = trq_MRSA[['ppid', 'DateIdentified', 'Does_the_patient_have_any_active_infection_prevention_and_control_alerts?_Code',
                      'Has_the_patient_transferred_from_any_ward_or_care_home_with_suspected_or_confirmed_Norovirus?_Code', 
                        'Does_the_patient_present_with_respiratory_symptoms_+/-_fever?_Code',
                      'Does_the_patient_have_a_rash_with_onset_in_the_last_24-48_hrs_+/-_fever_or_flu_like_symptoms?_Code',
       'Has_the_patient_been_in_contact_with_any_infectious_diseases?_Code']].fillna('N')
    for rc in ['Does_the_patient_have_any_active_infection_prevention_and_control_alerts?_Code',
                      'Has_the_patient_transferred_from_any_ward_or_care_home_with_suspected_or_confirmed_Norovirus?_Code', 
                        'Does_the_patient_present_with_respiratory_symptoms_+/-_fever?_Code',
                      'Does_the_patient_have_a_rash_with_onset_in_the_last_24-48_hrs_+/-_fever_or_flu_like_symptoms?_Code',
       'Has_the_patient_been_in_contact_with_any_infectious_diseases?_Code']:
        trq_MRSA[rc] = trq_MRSA[rc].map(cat_dict).astype(np.int8)
    trq_MRSA = trq_MRSA.rename(
        columns={'Does_the_patient_have_any_active_infection_prevention_and_control_alerts?_Code': 'trQ_mrsa_infection_prevention',
                'Has_the_patient_transferred_from_any_ward_or_care_home_with_suspected_or_confirmed_Norovirus?_Code': 'trQ_mrsa_transfer_with_norovirus',
                'Does_the_patient_present_with_respiratory_symptoms_+/-_fever?_Code': 'trQ_mrsa_resp_or_fever',
                'Does_the_patient_have_a_rash_with_onset_in_the_last_24-48_hrs_+/-_fever_or_flu_like_symptoms?_Code': 'trQ_mrsa_rash_fever_or_flu',
                'Has_the_patient_been_in_contact_with_any_infectious_diseases?_Code': 'trQ_mrsa_infectious_diseases_contact'})
    trq_MRSA['DateIdentified'] = pd.to_datetime(trq_MRSA['DateIdentified'])

    ### TRQ Nutritional Profile
    trq_nutr = trq_nutr[['ppid', 'DateIdentified', 'Does_the_patient_have_food_allergies?_Code',
       'Does_the_patient_have_any_swallowing_difficulties?_Code']].fillna('N')
    for rc in ['Does_the_patient_have_food_allergies?_Code',
       'Does_the_patient_have_any_swallowing_difficulties?_Code']:
        trq_nutr[rc] = trq_nutr[rc].map(cat_dict).astype(np.int8)
        
    trq_nutr = trq_nutr.rename(
        columns={'Does_the_patient_have_food_allergies?_Code': 'trQ_nutr_food_allergies',
                'Does_the_patient_have_any_swallowing_difficulties?_Code': 'trQ_nutr_swallowing_difficulty'})
    trq_nutr['DateIdentified'] = pd.to_datetime(trq_nutr['DateIdentified'])

    ### TRQ Bedrails
    trq_RUB = trq_RUB[['ppid', 'DateOfAssessment', 
                       'An_initial_documented_nursing_and_falls_risk_assessment_is_made_within_24_hours_of_admission_Code',
       'Is_the_patient_at_risk_of_falling_out_of_bed?_Code']].fillna('N')
    for rc in ['An_initial_documented_nursing_and_falls_risk_assessment_is_made_within_24_hours_of_admission_Code',
       'Is_the_patient_at_risk_of_falling_out_of_bed?_Code']:
        trq_RUB[rc] = trq_RUB[rc].map(cat_dict).astype(np.int8)
    trq_RUB = trq_RUB.rename(
        columns={'An_initial_documented_nursing_and_falls_risk_assessment_is_made_within_24_hours_of_admission_Code': 'trQ_rub_nursing_falls_risk_assessment',
                'Is_the_patient_at_risk_of_falling_out_of_bed?_Code': 'trQ_rub_at_risk_of_bed_fall'})
    trq_RUB['DateOfAssessment'] = pd.DatetimeIndex(pd.to_datetime(trq_RUB['DateOfAssessment'])).tz_localize(None)

    ### TRQ Pressure Ulcers (waterlow)
    trq_wt = trq_wt[['ppid', 'DateOfAssessment', 'Score']].rename(columns={'Score': 'trQ_waterlow_score'})
    trq_wt['DateOfAssessment'] = pd.DatetimeIndex(pd.to_datetime(trq_wt['DateOfAssessment'])).tz_localize(None)

    ### TRQ MUST score
    trq_MUST = trq_MUST[['ppid', 'DateOfAssessment', 'Total score']].rename(columns={'Total score': 'trQ_MUST_score'})
    trq_MUST['DateOfAssessment'] = pd.DatetimeIndex(pd.to_datetime(trq_MUST['DateOfAssessment'])).tz_localize(None)

    ### Pipeline
    full_coh = pd.DataFrame()
    inp_ch['HOSP_adt'] = pd.to_datetime(inp_ch['HOSP_adt'])
    inp_ch['AdmissionDate'] = pd.to_datetime(inp_ch['AdmissionDate'])
    for i in range(1, n_cohorts+1, window):
        print('Processing cohort:', i)
        cur_coh = inp_ch[inp_ch['coh_idx']==i]
        #pr_start = inp_ch[inp_ch['coh_idx']==i]['HOSP_iadate']
        trq_4at_ch = pd.merge(trq_4at, cur_coh[['ppid', 'HOSP_adt']], how='left', on='ppid')
        trq_4at_ch = trq_4at_ch[((trq_4at_ch['HOSP_adt'] + timedelta(hours=0)) >= trq_4at_ch['DateOfAssessment'])&(trq_4at_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        trq_4at_ch = trq_4at_ch.sort_values(['ppid', 'DateOfAssessment']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        trq_BBF_ch = pd.merge(trq_BBF, cur_coh[['ppid', 'AdmissionDate']], how='left', on='ppid')
        trq_BBF_ch = trq_BBF_ch[((trq_BBF_ch['AdmissionDate'] + timedelta(days=0)) >= trq_BBF_ch['DateIdentified'])&(trq_BBF_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        trq_BBF_ch = trq_BBF_ch.sort_values(['ppid', 'DateIdentified']).drop_duplicates(['ppid', 'AdmissionDate'], keep='last')
        trq_falls_ch = pd.merge(trq_falls, cur_coh[['ppid', 'HOSP_adt']], how='left', on='ppid')
        trq_falls_ch = trq_falls_ch[((trq_falls_ch['HOSP_adt'] + timedelta(hours=0)) >= trq_falls_ch['DateOfAssessment'])&(trq_falls_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        trq_falls_ch = trq_falls_ch.sort_values(['ppid', 'DateOfAssessment']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        trq_mobility_ch = pd.merge(trq_mobility, cur_coh[['ppid', 'HOSP_adt']], how='left', on='ppid')
        trq_mobility_ch = trq_mobility_ch[((trq_mobility_ch['HOSP_adt'] + timedelta(hours=0)) >= trq_mobility_ch['DateOfAssessment'])&(trq_mobility_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        trq_mobility_ch = trq_mobility_ch.sort_values(['ppid', 'DateOfAssessment']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        trq_MRSA_ch = pd.merge(trq_MRSA, cur_coh[['ppid', 'AdmissionDate']], how='left', on='ppid')
        trq_MRSA_ch = trq_MRSA_ch[((trq_MRSA_ch['AdmissionDate'] + timedelta(days=0)) >= trq_MRSA_ch['DateIdentified'])&(trq_MRSA_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        trq_MRSA_ch = trq_MRSA_ch.sort_values(['ppid', 'DateIdentified']).drop_duplicates(['ppid', 'AdmissionDate'], keep='last')
        trq_nutr_ch = pd.merge(trq_nutr, cur_coh[['ppid', 'AdmissionDate']], how='left', on='ppid')
        trq_nutr_ch = trq_nutr_ch[((trq_nutr_ch['AdmissionDate'] + timedelta(days=0)) >= trq_nutr_ch['DateIdentified'])&(trq_nutr_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        trq_nutr_ch = trq_nutr_ch.sort_values(['ppid', 'DateIdentified']).drop_duplicates(['ppid', 'AdmissionDate'], keep='last')
        trq_RUB_ch = pd.merge(trq_RUB, cur_coh[['ppid', 'HOSP_adt']], how='left', on='ppid')
        trq_RUB_ch = trq_RUB_ch[((trq_RUB_ch['HOSP_adt'] + timedelta(hours=0)) >= trq_RUB_ch['DateOfAssessment'])&(trq_RUB_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        trq_RUB_ch = trq_RUB_ch.sort_values(['ppid', 'DateOfAssessment']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        trq_wt_ch = pd.merge(trq_wt, cur_coh[['ppid', 'HOSP_adt']], how='left', on='ppid')
        trq_wt_ch = trq_wt_ch[((trq_wt_ch['HOSP_adt'] + timedelta(hours=0)) >= trq_wt_ch['DateOfAssessment'])&(trq_wt_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        trq_wt_ch = trq_wt_ch.sort_values(['ppid', 'DateOfAssessment']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        trq_MUST_ch = pd.merge(trq_MUST, cur_coh[['ppid', 'HOSP_adt']], how='left', on='ppid')
        trq_MUST_ch = trq_MUST_ch[((trq_MUST_ch['HOSP_adt'] + timedelta(hours=0)) >= trq_MUST_ch['DateOfAssessment'])&(trq_MUST_ch['ppid'].isin(inp_ch['ppid'].unique().tolist()))]
        trq_MUST_ch = trq_MUST_ch.sort_values(['ppid', 'DateOfAssessment']).drop_duplicates(['ppid', 'HOSP_adt'], keep='last')
        
        
        cur_coh = pd.merge(cur_coh, trq_4at_ch.drop(['DateOfAssessment'], axis=1), how='left', on=['ppid', 'HOSP_adt'])
        cur_coh = pd.merge(cur_coh, trq_BBF_ch.drop(['DateIdentified'], axis=1), how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, trq_falls_ch.drop(['DateOfAssessment'], axis=1), how='left', on=['ppid', 'HOSP_adt'])
        cur_coh = pd.merge(cur_coh, trq_mobility_ch.drop(['DateOfAssessment'], axis=1), how='left', on=['ppid', 'HOSP_adt'])
        cur_coh = pd.merge(cur_coh, trq_nutr_ch.drop(['DateIdentified'], axis=1), how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, trq_MRSA_ch.drop(['DateIdentified'], axis=1), how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, trq_RUB_ch.drop(['DateOfAssessment'], axis=1), how='left', on=['ppid', 'HOSP_adt'])
        cur_coh = pd.merge(cur_coh, trq_wt_ch.drop(['DateOfAssessment'], axis=1), how='left', on=['ppid', 'HOSP_adt'])
        cur_coh = pd.merge(cur_coh, trq_MUST_ch.drop(['DateOfAssessment'], axis=1), how='left', on=['ppid', 'HOSP_adt'])
        ## Fill NAs
        for col in ['trQ_bwm_urinary_catheterisation',
       'trQ_bwm_urinary_incontinence', 'trQ_bwm_dysuria',
       'trQ_bwm_>6times_per_day', 'trQ_bwm_nocturia_>2_per_night',
       'trQ_bwm_faeces_incontinence', 'trQ_bwm_constipation',
       'trQ_bwm_diarrhoea', 'trQ_bwm_blood_in_stools', 'trQ_bwm_medication',
       'trQ_falls_within_6_months', 'trQ_falls_clinical_risk', 'trQ_nutr_food_allergies',
       'trQ_nutr_swallowing_difficulty', 'trQ_mrsa_infection_prevention',
       'trQ_mrsa_transfer_with_norovirus', 'trQ_mrsa_resp_or_fever',
       'trQ_mrsa_rash_fever_or_flu', 'trQ_mrsa_infectious_diseases_contact',
       'trQ_rub_nursing_falls_risk_assessment', 'trQ_rub_at_risk_of_bed_fall']:
            cur_coh[col] = cur_coh[col].fillna(0).astype(np.int8)

        for col in ['trQ_mobility_walking', 'trQ_mobility_toileting',
       'trQ_mobility_bathing', 'trQ_mobility_bed_rolling',
       'trQ_mobility_bed_moveup', 'trQ_mobility_bed_out',
       'trQ_mobility_bed_in', 'trQ_mobility_sss', 'trQ_mobility_lateral',
       'trQ_mobility_floorup']:
            cur_coh[col] = cur_coh[col].fillna('NA').astype('category')
            cur_coh = pd.get_dummies(cur_coh, columns=[col])

        for col in ['trQ_4at', 'trQ_waterlow_score', 'trQ_MUST_score']:
            cur_coh[col] = cur_coh[col].fillna(-1).astype(np.int16)

        for col in cur_coh.columns:
            if 'trQ_mobility' in col:
                cur_coh[col] = cur_coh[col].astype(np.int8)
        
        full_coh = pd.concat([full_coh, cur_coh], axis=0)

    return full_coh

In [ ]:
inp_ch = get_trakq_features(inp_ch, trq_4at, trq_BBF, trq_falls, trq_mobility,
                   trq_MRSA, trq_MUST, trq_nutr, trq_RUB, trq_wt)

In [ ]:
inp_ch.shape

In [ ]:
inp_ch.columns.tolist()

In [ ]:
for col in inp_ch.columns:
    if 'trQ_' in col:
        print(inp_ch[col].value_counts())

In [ ]:
inp_ch.shape, inp_ch.ppid.nunique()

In [ ]:
inp_ch.to_csv('', index=False)

#### Lab tests

In [ ]:
inp_ch = pd.read_csv('')

In [ ]:
# Get chunk size split for loading data into batches
total_rows = sum(1 for _ in open('')) - 1
chunk_size = total_rows // 4
print(chunk_size)

In [ ]:
c_idx = 0
term = 0
for chunk in pd.read_csv('', sep='\t', low_memory=False,
                        chunksize=chunk_size):
    if c_idx == term:
        labs_data = chunk
        break
    c_idx += 1

In [ ]:
labs_data.shape

In [ ]:
labs_data.columns

In [ ]:
labs_data.Status.value_counts()

In [ ]:
## Correct eGFR
labs_data['RangeMin'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['RangeMin'].isnull()), '60.0',
                                 labs_data['RangeMin'])
labs_data['RangeMax'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['RangeMax'].isnull()), '150.0',
                                 labs_data['RangeMax'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='>60'), '61.0',
                                 labs_data['Value'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='>60.0'), '61.0',
                                 labs_data['Value'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='Not Indicated'), '61.0',
                                 labs_data['Value'])

In [ ]:
### Helper-functions
def adjust_values(val):
    try:
        if '>' in val:
            return float(val[1:]) + 1
        elif '<' in val:
            return float(val[1:]) - 1
        else:
            return val
    except ValueError:
        return val

def get_normal_values_h(bloods_data):
    if bloods_data.Value > bloods_data.RangeMax:
        return 1
    else:
        return 0

def get_normal_values_l(bloods_data):
    if bloods_data.Value < bloods_data.RangeMin:
        return 1
    else:
        return 0

def prep_labs(lab_set, inp_ch):
    labs_sel = lab_set[lab_set.ppid.isin(inp_ch.ppid.unique())]
    labs_sel = labs_sel[labs_sel['Status'] == 'A']
    labs_sel = labs_sel[['ppid', 'SpecimenCollectionDate', 'SpecimenCollectionTime', 'Value', 'TestItem', 'RangeMin', 'RangeMax']]
    labs_sel['SpecimenCollectionDate'] = pd.to_datetime(labs_sel['SpecimenCollectionDate'].astype(str) + ' ' + labs_sel['SpecimenCollectionTime'].astype(str), 
                                                        errors='coerce')
    labs_sel = labs_sel[labs_sel.Value.notnull()]
    labs_sel['Value'] = labs_sel['Value'].apply(adjust_values)
    labs_sel['Value'] = pd.to_numeric(labs_sel['Value'], errors='coerce')
    labs_sel['RangeMin'] = pd.to_numeric(labs_sel['RangeMin'], errors='coerce')
    labs_sel['RangeMax'] = pd.to_numeric(labs_sel['RangeMax'], errors='coerce')
    labs_sel['TestItem'] = labs_sel['TestItem'].astype(str).str.strip()
    print(labs_sel.shape)
    labs_sel.dropna(subset=['ppid', 'SpecimenCollectionDate', 'SpecimenCollectionTime', 'TestItem'], inplace=True)
    print(labs_sel.shape)
    return labs_sel

def get_lab_features(labs_sel, inp_ch, n_cohorts=10, window=15, 
                     day_col='_days', rec_col='_recorded', nor_col='_normal_low', nor_col2='_normal_high',
                    val_col='_value', rm_col='_rmean', rs_col='_rstd'):
    ### Pipeline
    full_coh = pd.DataFrame()
    inp_ch['HOSP_adt'] = pd.to_datetime(inp_ch['HOSP_adt'])
    inp_ch['AdmissionDate'] = pd.to_datetime(inp_ch['AdmissionDate'])
    for i in range(1, n_cohorts+1, window):
        print('Processing cohort:', i)
        cur_coh = inp_ch[inp_ch['coh_idx']==i]
        for test_item in labs_sel.TestItem.unique():
            print('Processing features for:', test_item)
            labs_sel_test = labs_sel[labs_sel['TestItem'] == test_item]
            labs_sel_test = pd.merge(labs_sel_test, cur_coh[['ppid', 'HOSP_adt']], how='left', on='ppid')
            labs_sel_test = labs_sel_test[((labs_sel_test['HOSP_adt'] + timedelta(hours=0)) >= labs_sel_test['SpecimenCollectionDate'])&(labs_sel_test.ppid.isin(cur_coh.ppid.unique()))]
            #### Simple moving average over 365 days
            labs_sel_test_t = labs_sel_test[labs_sel_test['SpecimenCollectionDate'] >= (labs_sel_test['HOSP_adt'] - timedelta(days=365))]
            rstd = pd.DataFrame(labs_sel_test_t.groupby(['ppid'])['Value'].std()).rename(columns={'Value':'rstd'})
            rmean = pd.DataFrame(labs_sel_test_t.groupby(['ppid'])['Value'].mean()).rename(columns={'Value':'rmean'})
            labs_sel_test = pd.merge(labs_sel_test, rmean, how='left', on=['ppid'])
            labs_sel_test = pd.merge(labs_sel_test, rstd, how='left', on=['ppid'])
            #print(labs_sel_test.head())
            labs_sel_test = labs_sel_test.sort_values(by=['ppid', 'SpecimenCollectionDate'], ascending=[True, False]).groupby(['ppid', 'HOSP_adt']).first().reset_index()
            labs_sel_test['days_since_test'] = ((labs_sel_test['HOSP_adt'] + timedelta(hours=0)) - labs_sel_test['SpecimenCollectionDate']).dt.days.astype(np.int32)
            labs_sel_test['recorded'] = np.where(labs_sel_test['Value'].isnull(), 0, 1)
            labs_sel_test['normal_l'] = labs_sel_test.apply(get_normal_values_l, axis=1)
            labs_sel_test['normal_h'] = labs_sel_test.apply(get_normal_values_h, axis=1)
            print('Number of pts with test:', labs_sel_test[labs_sel_test.recorded==1].ppid.nunique())
            labs_sel_test.rename(columns={'days_since_test': test_item.replace(' ', '_').lower() + day_col,
                                          'Value': test_item.replace(' ', '_').lower() + val_col,
                                          'recorded': test_item.replace(' ', '_').lower() + rec_col,
                                          'normal_l': test_item.replace(' ', '_').lower() + nor_col,
                                         'normal_h': test_item.replace(' ', '_').lower() + nor_col2,
                                         'rmean': test_item.replace(' ', '_').lower() + rm_col,
                                         'rstd': test_item.replace(' ', '_').lower() + rs_col}, inplace=True)
            labs_sel_test = labs_sel_test.drop(['SpecimenCollectionDate', 'SpecimenCollectionTime',
                                                'TestItem', 'RangeMin', 'RangeMax'], axis=1)
            cur_coh = pd.merge(cur_coh, labs_sel_test, on=['ppid', 'HOSP_adt'], how='left')
            
        full_coh = pd.concat([full_coh, cur_coh], axis=0)
        full_coh[list(col for col in full_coh.columns if 'days' in col)] = full_coh[list(col for col in full_coh.columns if 'days' in col)].fillna(9999).astype(np.int32)
        full_coh[list(col for col in full_coh.columns if 'value' in col)] = full_coh[list(col for col in full_coh.columns if 'value' in col)].fillna(-9999.0).astype(np.float32)
        full_coh[list(col for col in full_coh.columns if 'rmean' in col)] = full_coh[list(col for col in full_coh.columns if 'rmean' in col)].fillna(-9999.0).astype(np.float32)
        full_coh[list(col for col in full_coh.columns if 'rstd' in col)] = full_coh[list(col for col in full_coh.columns if 'rstd' in col)].fillna(-9999.0).astype(np.float32)
        full_coh[list(col for col in full_coh.columns if 'normal' in col)] = full_coh[list(col for col in full_coh.columns if 'normal' in col)].fillna(0).astype(np.int8)
        full_coh[list(col for col in full_coh.columns if 'recorded' in col)] = full_coh[list(col for col in full_coh.columns if 'recorded' in col)].fillna(0).astype(np.int8)
        return full_coh

In [ ]:
labs_sel = prep_labs(labs_data, inp_ch)
inp_ch = get_lab_features(labs_sel, inp_ch)

In [ ]:
inp_ch.shape, inp_ch.ppid.nunique()

In [ ]:
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)

In [ ]:
### Process chunk 2
c_idx = 0
term = 1
for chunk in pd.read_csv('', sep='\t', low_memory=False,
                        chunksize=chunk_size):
    print('Processing chunk:', c_idx)
    if c_idx == term:
        labs_data = chunk
        break
    c_idx += 1

In [ ]:
## Correct eGFR
labs_data['RangeMin'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['RangeMin'].isnull()), '60.0',
                                 labs_data['RangeMin'])
labs_data['RangeMax'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['RangeMax'].isnull()), '150.0',
                                 labs_data['RangeMax'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='>60'), '61.0',
                                 labs_data['Value'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='>60.0'), '61.0',
                                 labs_data['Value'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='Not Indicated'), '61.0',
                                 labs_data['Value'])

In [ ]:
labs_sel = prep_labs(labs_data, inp_ch)
inp_ch = get_lab_features(labs_sel, inp_ch, rec_col='_recorded2', nor_col='_normal_low2', nor_col2='_normal_high2',
                          val_col='_value2', day_col='_days2', rm_col='_rmean2', rs_col='_rstd2')

In [ ]:
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)

In [ ]:
### Process chunk 3
c_idx = 0
term = 2
for chunk in pd.read_csv('', sep='\t', low_memory=False,
                        chunksize=chunk_size):
    print('Processing chunk:', c_idx)
    if c_idx == term:
        labs_data = chunk
        break
    c_idx += 1
## Correct eGFR
labs_data['RangeMin'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['RangeMin'].isnull()), '60.0',
                                 labs_data['RangeMin'])
labs_data['RangeMax'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['RangeMax'].isnull()), '150.0',
                                 labs_data['RangeMax'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='>60'), '61.0',
                                 labs_data['Value'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='>60.0'), '61.0',
                                 labs_data['Value'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='Not Indicated'), '61.0',
                                 labs_data['Value'])
labs_sel = prep_labs(labs_data, inp_ch)
inp_ch = get_lab_features(labs_sel, inp_ch, rec_col='_recorded3', nor_col='_normal_low3', nor_col2='_normal_high3', val_col='_value3',
                         day_col='_days3', rm_col='_rmean3', rs_col='_rstd3')

In [ ]:
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)

In [ ]:
### Process chunk 4
#labs_data = pd.read_csv('', sep='\t', low_memory=False)
c_idx = 0
term = 3
for chunk in pd.read_csv('', sep='\t', low_memory=False,
                        chunksize=chunk_size):
    print('Processing chunk:', c_idx)
    if c_idx == term:
        labs_data = chunk
        break
    c_idx += 1
## Correct eGFR
labs_data['RangeMin'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['RangeMin'].isnull()), '60.0',
                                 labs_data['RangeMin'])
labs_data['RangeMax'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['RangeMax'].isnull()), '150.0',
                                 labs_data['RangeMax'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='>60'), '61.0',
                                 labs_data['Value'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='>60.0'), '61.0',
                                 labs_data['Value'])
labs_data['Value'] = np.where((labs_data['TestItem'] == 'eGFR (/1.73m2)')&(labs_data['Value']=='Not Indicated'), '61.0',
                                 labs_data['Value'])
labs_sel = prep_labs(labs_data, inp_ch)
inp_ch = get_lab_features(labs_sel, inp_ch, rec_col='_recorded4', nor_col='_normal_low4', nor_col2='_normal_high4', val_col='_value4',
                         day_col='_days4', rm_col='_rmean4', rs_col='_rstd4')

In [ ]:
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)

In [ ]:
inp_ch = pd.read_csv('')

In [ ]:
inp_ch.columns.tolist()[::-1]

In [ ]:
### Merge lab test chunk values
bloods_names = [col.rsplit('_', 1)[0] for col in inp_ch.columns if '_recorded' in col]
bloods_names = list(set(bloods_names))
#print(bloods_names, len(bloods_names))
blood_columns_r = {bl_name: [col for col in inp_ch.columns if f"{bl_name}_recorded" in col] for bl_name in bloods_names}
blood_columns_d = {bl_name: [col for col in inp_ch.columns if f"{bl_name}_days" in col] for bl_name in bloods_names}
blood_columns_v = {bl_name: [col for col in inp_ch.columns if f"{bl_name}_value" in col] for bl_name in bloods_names}
blood_columns_n1 = {bl_name: [col for col in inp_ch.columns if f"{bl_name}_normal_low" in col] for bl_name in bloods_names}
blood_columns_n2 = {bl_name: [col for col in inp_ch.columns if f"{bl_name}_normal_high" in col] for bl_name in bloods_names}
blood_columns_rm = {bl_name: [col for col in inp_ch.columns if f"{bl_name}_rmean" in col] for bl_name in bloods_names}
blood_columns_rstd = {bl_name: [col for col in inp_ch.columns if f"{bl_name}_rstd" in col] for bl_name in bloods_names}

def merge_labs_n(b_data, cols):
    return np.nanmax(b_data[cols].values, axis=1)
    
#def merge_labs_n(b_data, cols):
    #b_sel = b_data[cols].values
    #b_sel = np.where(b_sel!=0, b_sel, np.nan)
    #return np.nan_to_num(np.nanmax(b_sel, axis=1), nan=0)
           
for bl_name in tqdm(bloods_names):
    print('Processing ', bl_name)
    col_r = blood_columns_r[bl_name]
    col_d = blood_columns_d[bl_name]
    col_v = blood_columns_v[bl_name]
    col_n1 = blood_columns_n1[bl_name]
    col_n2 = blood_columns_n2[bl_name]
    col_rm = blood_columns_rm[bl_name]
    col_rstd = blood_columns_rstd[bl_name]
    
    if col_r:
        inp_ch[bl_name + '_r'] = merge_labs_n(inp_ch, col_r)
        print('recorded value')
    if col_d:
        inp_ch[bl_name + '_d'] = merge_labs_n(inp_ch, col_d)
        print('days value')
    if col_v:
        inp_ch[bl_name + '_v'] = merge_labs_n(inp_ch, col_v)
        print('test value')
    if col_n1:
        inp_ch[bl_name + '_nl'] = merge_labs_n(inp_ch, col_n1)
        print('normal low value')
    if col_n2:
        inp_ch[bl_name + '_nh'] = merge_labs_n(inp_ch, col_n2)
        print('normal high value')
    if col_rm:
        inp_ch[bl_name + '_rm'] = merge_labs_n(inp_ch, col_rm)
        print('rolling mean value')
    if col_rstd:
        inp_ch[bl_name + '_rs'] = merge_labs_n(inp_ch, col_rstd)
        print('rolling std value')
        
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)

In [ ]:
inp_ch = inp_ch.drop(inp_ch[[col for col in inp_ch if '_recorded' in col]].columns.tolist(), axis=1)
inp_ch = inp_ch.drop(inp_ch[[col for col in inp_ch if '_normal_low' in col]].columns.tolist(), axis=1)
inp_ch = inp_ch.drop(inp_ch[[col for col in inp_ch if '_normal_high' in col]].columns.tolist(), axis=1)
inp_ch = inp_ch.drop(inp_ch[[col for col in inp_ch if '_value' in col]].columns.tolist(), axis=1)
inp_ch = inp_ch.drop(inp_ch[[col for col in inp_ch if '_rstd' in col]].columns.tolist(), axis=1)
inp_ch = inp_ch.drop(inp_ch[[col for col in inp_ch if '_rmean' in col]].columns.tolist(), axis=1)
inp_ch = inp_ch.drop(inp_ch[[col for col in inp_ch if '_days' in col and not 'vent_days' in col]].columns.tolist(), axis=1)

In [ ]:
inp_ch.shape

In [ ]:
for col in inp_ch[list(col for col in inp_ch.columns if '_nl' in col)]:
    print(inp_ch.groupby('coh_idx')[col].value_counts())

In [ ]:
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='ast:alt_ratio_nl', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='procalcitonin_nl', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='hs_troponin_t_nl', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='ast_nl', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='ast:alt_ratio_nl', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='c-reactive_prot_nl', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='ntprobnp_nl', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='procalcitonin_nh', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='egfr_(/1.73m2)_nh', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='ntprobnp_nh', axis=1).columns, axis=1)

In [ ]:
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)

#### Prescribing

In [ ]:
inp_ch = pd.read_csv('')

In [ ]:
### Chunk 1
presc_data = pd.read_csv('', sep='\t', low_memory=False)

In [ ]:
presc_data.head()

In [ ]:
presc_data['PI BNF Section Description'].value_counts()

In [ ]:
def prep_pis(presc_data, inp_ch, sample_upper=1000):
    PIS_sel = presc_data[['ppid', 'Paid Date', 'PI Approved Name', 'PI BNF Section Description']]
    PIS_sel = PIS_sel[PIS_sel.ppid.isin(inp_ch.ppid.unique())]
    PIS_sel['Paid Date'] = pd.to_datetime(PIS_sel['Paid Date'])
    PIS_sel['PI Approved Name'] = PIS_sel['PI Approved Name'].astype(str).str.strip()
    PIS_sel['PI BNF Section Description'] = np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'LIPID-REGULATING DRUGS', 'lipid_regulators',
                                                     np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'DIURETICS', 'diuretics',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'HYPERTENSION AND HEART FAILURE', 'anti_hypertension_hf_drugs',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'ANTIDEPRESSANT DRUGS', 'antidepressant_drugs',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'NITRATES ,CALCIUM-CHANNEL BLOCKERS AND OTHER ANTIANGINAL DRUGS', 'nitrates_ccb_drugs',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'BETA-ADRENOCEPTOR BLOCKING DRUGS', 'beta_blockers',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'ANTIPLATELET DRUGS', 'antiplatelet_drugs',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'DRUGS AFFECTING BONE METABOLISM', 'bone_metabolism_affecting_drugs',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'ANTICOAGULANTS AND PROTAMINE', 'anticoagulant_protamine_drugs',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'DRUGS USED IN PSYCHOSES AND RELATED DISORDERS', 'antipsychotics',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'DRUGS FOR DEMENTIA', 'antidementia_drugs',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'DRUGS USED IN NAUSEA AND VERTIGO', 'nausea_vertigo_drugs',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'DRUGS USED IN PARKINSONISM AND RELATED DISORDERS', 'parkinsonism_drugs',
                                                    np.where(PIS_sel['PI BNF Section Description'].str.strip() == 'DRUGS FOR GENITO-URINARY DISORDERS', 'genito_urinary_drugs', 'other'))))))))))))))
    PIS_sel = PIS_sel[PIS_sel['PI BNF Section Description'] != 'other']                                 
    PIS_sel = PIS_sel.dropna()
    print(PIS_sel.shape)
    #top_drugs = PIS_sel.groupby('PI Approved Name')['ppid'].count().reset_index(name='count').sort_values('count', ascending=False).head(60)
    return PIS_sel

def rename(col):
    if isinstance(col, tuple):
        col = '_'.join(str(c) for c in col)
    return col

def get_pis_features(pis_sel, inp_ch, n_cohorts=10, window=15):
    ### Pipeline
    full_coh = pd.DataFrame()
    inp_ch['AdmissionDate'] = pd.to_datetime(inp_ch['AdmissionDate'])
    for i in range(1, n_cohorts+1, window):
        print('Processing cohort:', i)
        cur_coh = inp_ch[inp_ch['coh_idx']==i]
        pis_ch = pd.merge(pis_sel, cur_coh[['ppid', 'AdmissionDate']], how='left', on='ppid')
        pis_ch = pis_ch[((pis_ch['AdmissionDate'] + timedelta(days=0)) > pis_ch['Paid Date'])&(pis_ch.ppid.isin(cur_coh.ppid.unique()))]
        ph_drug_ids = pis_ch.groupby(['ppid', 'AdmissionDate', 'PI BNF Section Description']).size().reset_index(name='n_presc')
        print(ph_drug_ids.shape)
        pis_ch = pis_ch.sort_values(['ppid', 'AdmissionDate', 'Paid Date', 'PI BNF Section Description'])
        ph_drug_mind = pis_ch.groupby(['ppid', 'PI BNF Section Description', 'AdmissionDate'])['Paid Date'].min().reset_index(name='first_date')
        ph_drug_maxd = pis_ch.groupby(['ppid', 'PI BNF Section Description', 'AdmissionDate'])['Paid Date'].max().reset_index(name='last_date')
        ph_drug_mind['dsf'] = ((ph_drug_mind['AdmissionDate'] + timedelta(days=0)) - ph_drug_mind['first_date']).dt.days
        ph_drug_maxd['dsl'] = ((ph_drug_maxd['AdmissionDate'] + timedelta(days=0)) - ph_drug_maxd['last_date']).dt.days
        ph_drug_ids = pd.merge(ph_drug_ids, ph_drug_mind[['ppid', 'AdmissionDate', 'PI BNF Section Description', 'dsf']], how='left', on=['ppid', 'AdmissionDate', 
                                                                                                                                          'PI BNF Section Description'])
        ph_drug_ids = pd.merge(ph_drug_ids, ph_drug_maxd[['ppid', 'AdmissionDate', 'PI BNF Section Description', 'dsl']], how='left', on=['ppid', 'AdmissionDate', 
                                                                                                                                          'PI BNF Section Description'])
        print(ph_drug_ids.shape)
        ph_drug_ids['PI BNF Section Description'] = ph_drug_ids['PI BNF Section Description'].astype(str).replace(' ', '_')
        #print(ph_drug_ids.head(10))
        ph_drugs_piv = ph_drug_ids.set_index(['ppid', 'AdmissionDate', 'PI BNF Section Description']).unstack()
        ph_drugs_piv.columns = map(rename, ph_drugs_piv)
        ph_drugs_piv = ph_drugs_piv.reset_index()
        ph_drugs_total = ph_drug_ids.groupby(['ppid', 'AdmissionDate'])['PI BNF Section Description'].nunique().reset_index(name='total_drug_categories')
        cur_coh = pd.merge(cur_coh, ph_drugs_piv, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, ph_drugs_total, how='left', on=['ppid', 'AdmissionDate'])
        full_coh = pd.concat([full_coh, cur_coh], axis=0)

    days_cols = list(col for col in full_coh.columns if 'dsf' in col or 'dsl' in col)
    nums_cols = list(col for col in full_coh.columns if 'n_presc' in col)
    full_coh[days_cols] = full_coh[days_cols].fillna(9999).astype(np.int32)
    full_coh[nums_cols] = full_coh[nums_cols].fillna(0).astype(np.int16)
    full_coh['total_drug_categories'] = full_coh['total_drug_categories'].fillna(0).astype(np.int8)
    return full_coh

In [ ]:
pis_sel = prep_pis(presc_data, inp_ch)
inp_ch = get_pis_features(pis_sel, inp_ch)

In [ ]:
#inp_ch.columns.tolist()[401:]

In [ ]:
#tgt_ch = inp_ch.columns[401:]
inp_ch = inp_ch.rename(columns={c:c+'_ch1' for c in inp_ch.columns if c in tgt_ch})

In [ ]:
### Chunk 2
presc_data = pd.read_csv('', sep='\t', low_memory=False)
pis_sel = prep_pis(presc_data, inp_ch)
inp_ch = get_pis_features(pis_sel, inp_ch)

In [ ]:
#inp_ch.columns.tolist()[444:]

In [ ]:
#tgt_ch = inp_ch.columns[444:]
inp_ch = inp_ch.rename(columns={c:c+'_ch2' for c in inp_ch.columns if c in tgt_ch})

In [ ]:
### Merge lab test chunk values
dr_names = ["_".join(col.split('_')[2:-1]) for col in inp_ch.columns if 'n_presc' in col]
dr_names = list(set(dr_names)) + ['total_drug_categories']
#print(bloods_names, len(bloods_names))
dr_columns_v = {dr_name: [col for col in inp_ch.columns if f"n_presc_{dr_name}" in col] for dr_name in dr_names}
dr_columns_df = {dr_name: [col for col in inp_ch.columns if f"dsf_{dr_name}" in col] for dr_name in dr_names}
dr_columns_dl = {dr_name: [col for col in inp_ch.columns if f"dsl_{dr_name}" in col] for dr_name in dr_names}
dr_columns_c = {'total_drug_categories': ['total_drug_categories_ch1', 'total_drug_categories_ch2']}

def merge_dr_v(b_data, cols):
    return np.nansum(b_data[cols].values, axis=1)

def merge_dr_df(b_data, cols):
    return np.nanmax(b_data[cols].values, axis=1)

def merge_dr_dl(b_data, cols):
    return np.nanmax(b_data[cols].values, axis=1)
    
def merge_dr_c(b_data, cols):
    return np.nanmax(b_data[cols].values, axis=1)
           
for dr_name in tqdm(dr_names):
    print('Processing ', dr_name)
    col_v = dr_columns_v[dr_name]
    col_df = dr_columns_df[dr_name]
    col_dl = dr_columns_dl[dr_name]
    if dr_name == 'total_drug_categories':
        col_c = dr_columns_c[dr_name]
        inp_ch['total_drug_categories'] = merge_labs_n(inp_ch, col_c).astype(np.int8)
        print('categories value') 
    else:
        if col_v:
            inp_ch['n_presc_' + dr_name] = merge_dr_v(inp_ch, col_v)
            print('drug value')
        if col_df:
            inp_ch['dsf_' + dr_name] = merge_dr_df(inp_ch, col_df)
            print('days first value')
        if col_dl:
            inp_ch['dsl_' + dr_name] = merge_dr_dl(inp_ch, col_dl)
            print('days last value')
        
        
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)

In [ ]:
for col in inp_ch[list(col for col in inp_ch.columns if 'n_presc' in col)]:
    print(inp_ch.groupby('coh_idx')[col].value_counts())

In [ ]:
inp_ch = inp_ch.drop([col for col in inp_ch.columns if col.endswith('_ch1') or col.endswith('_ch2')], axis=1)
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)

In [ ]:
inp_ch.shape, inp_ch.ppid.nunique()

In [ ]:
inp_ch.columns.tolist()

#### GP/SMR READ phenotype history

In [ ]:
inp_ch = pd.read_csv('')

In [ ]:
smr_data = pd.read_csv('', sep='\t', low_memory=False, encoding='iso-8859-1')
gp_data = pd.read_csv('', sep='\t', low_memory=False, encoding='iso-8859-1')

In [ ]:
smr_data.PhenotypeGroup.value_counts()

In [ ]:
print(gp_data[gp_data.PhenotypeGroup.str.lower().str.contains('genit')]['PhenotypeName'].value_counts())
print(len(smr_data[smr_data.PhenotypeName.str.lower().str.contains('arth')]['PhenotypeName'].value_counts()))

In [ ]:
def prep_pheno_features(cm_data, inp_ch, max_events=1, fn='', ctype='ReadCode'):
    READ_sel = cm_data[['ppid', 'EventDate', ctype, 'PhenotypeName', 'PhenotypeCategory', 'PhenotypeGroup']]
    READ_sel = READ_sel[READ_sel.ppid.isin(inp_ch.ppid.unique())]
    READ_sel['EventDate'] = pd.to_datetime(pd.to_datetime(READ_sel['EventDate']).dt.date)
    READ_sel[ctype] = READ_sel[ctype].astype(str).str.strip()
    READ_sel['PhenotypeName'] = READ_sel['PhenotypeName'].astype(str).str.strip()
    READ_sel = READ_sel.dropna()
    print(READ_sel.shape)
    READ_types = READ_sel[['ppid', 'PhenotypeName']].groupby(['PhenotypeName']).size().reset_index(name='num_events')
    READ_types = READ_types[READ_types['num_events'] > max_events].sort_values('num_events', ascending=False)
    to_repl = [' ', '.', ':', '(', ')']
    for rep in to_repl:
        READ_types['ph_code'] = READ_types['PhenotypeName'].str.replace(rep, ' ')
    READ_types['ph_code'] = READ_types['ph_code'].replace(' ', '_', regex=True).str.lower()
    READ_types['ph_code'] = READ_types['ph_code'].str.replace('\W', '_', regex=True).str.replace('__', '_', regex=True).str.rstrip('_')
    print(READ_sel.shape)
    print(READ_types.shape)
    READ_sel = READ_sel[READ_sel['PhenotypeName'].isin(READ_types['PhenotypeName'].unique())]
    READ_sel = pd.merge(READ_sel, READ_types[['PhenotypeName', 'ph_code']], how='left', on='PhenotypeName')
    READ_sel.drop(columns=[ctype], axis=1, inplace=True)
    ### Build custom comorbidity fields as input features
    READ_sel['ph_code'] = np.where((READ_sel['PhenotypeName'].str.lower().str.contains('primary malignancy'))|
                                        (READ_sel['PhenotypeName'].str.lower().str.contains('secondary malignancy'))|
                                        READ_sel['PhenotypeName'].str.contains('|'.join(['Leukaemia', 'Hodgkin Lymphoma', 
                                                                           'Multiple myeloma and malignant plasma cell neoplasms',
                                                                           'Myelodysplastic syndromes', 'Non-Hodgkin Lymphoma'])), 
                                   'physltc_historical_or_active_cancer', READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Abdominal aortic aneurysm', 'physltc_abd_aortic_aneurysm', READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].str.contains('|'.join(['Coronary heart disease not otherwise specified',
                                                                                                              'Myocardial infarction', 
                                                                                                              'Stable angina',
                                                                                                              'Unstable Angina'])), 
                                   'physltc_ischaemic_heart_disease', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].isin(['Hypertension']), 'physltc_hypertension', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].str.contains('|'.join(['Ischaemic stroke', 'Stroke NOS',
                                                                                            'Intracerebral haemorrhage', 
                                                                                    'Subarachnoid haemorrhage',
                                                                                    'Transient ischaemic attack'])), 'physltc_stroke', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Peripheral arterial disease', 'physltc_per_vascular_disease', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].isin(['Crohn\'s disease', 'Ulcerative colitis']), 'physltc_inf_bowel_disease', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].isin(['Liver fibrosis, sclerosis and cirrhosis',
                                                                  'Hepatic failure', 'Oesophageal varices', 'Portal hypertension']), 
                                   'physltc_liver_disease', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].isin(['Diabetes', 'Diabetic ophthalmic complications',
                                                                  'Diabetic neurological complications']), 'physltc_diabetes', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].isin(['End stage renal disease', 'Glomerulonephritis']), 'physltc_chronic_renal_disease', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].isin(['Osteoarthritis (excl spine)', 'Rheumatoid Arthritis',
                                                                  'Psoriatic arthropathy', 'Juvenile arthritis',
                                                                  'Enteropathic arthropathy','Ankylosing spondylitis',
                                                                  'Spondylolisthesis']), 'physltc_arthritis_arthropathy', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Fracture of hip', 'physltc_hip_fracture', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].isin(['Dementia', 'Parkinson\'s disease',
                                                                  'Motor neuron disease']), 'physltc_prog_neur_disease', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Chronic Obstructive Pulmonary Disease (COPD)', 'physltc_copd', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Other interstitial pulmonary diseases with fibrosis', 
                                   'physltc_pulmonary_fibrosis', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Asthma', 
                                   'physltc_asthma', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Bronchiectasis', 
                                   'physltc_bronchiectasis', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Epilepsy', 
                                   'physltc_epilepsy', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Osteoporosis', 
                                   'physltc_osteoporosis', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Obesity', 
                                   'physltc_obesity', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Heart failure', 
                                   'physltc_heart_failure', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Atrial fibrillation', 
                                   'physltc_atrial_fibrillation', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].isin(['Alcohol Problems', 'Other psychoactive substance misuse', 
                                                                  'Alcoholic liver disease']), 
                                   'menltc_alcohol_substance_misuse', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName'].isin(['Schizophrenia, schizotypal and delusional disorders', 
                                                                   'Bipolar affective disorder and mania',
                                                                  'Autism and Asperger\'s syndrome',
                                                                  'Anorexia and bulimia nervosa']), 
                                   'menltc_chronic_psychiatry_disorder', 
                                   READ_sel['ph_code'])
    READ_sel['ph_code'] = np.where(READ_sel['PhenotypeName']=='Depression', 
                                   'menltc_depression', 
                                   READ_sel['ph_code'])
    print(READ_sel.shape)
    READ_types = pd.merge(READ_types, READ_sel[['PhenotypeName', 'ph_code']].rename(columns={'ph_code':
                                                                                             'ltc_code'}), how='left', on='PhenotypeName').drop_duplicates()
    READ_types.to_csv(fn, index=False)
    return READ_sel

def get_pheno_features(cm_data, inp_ch, n_cohorts=10, window=15, tn='total_GP_cmr',
                       dfn='dsf_GP_diag', dln='dsl_GP_diag', gn='GP_diag'):
    ### Pipeline
    full_coh = pd.DataFrame()
    inp_ch['HOSP_adt'] = pd.to_datetime(inp_ch['HOSP_adt'])
    inp_ch['AdmissionDate'] = pd.to_datetime(inp_ch['AdmissionDate'])
    for i in range(1, n_cohorts+1, window):
        print('Processing cohort:', i)
        cur_coh = inp_ch[inp_ch['coh_idx']==i]
        cm_ch = pd.merge(cm_data, cur_coh[['ppid', 'AdmissionDate']], how='left', on='ppid')
        cm_ch = cm_ch[((cm_ch['AdmissionDate'] + timedelta(days=0)) > cm_ch['EventDate'])&(cm_ch.ppid.isin(cur_coh.ppid.unique()))]
        READ_sum = cm_ch.groupby(['ppid', 'AdmissionDate']).size().reset_index(name=tn)
        #print(cm_ch.head())
        READ_ids = cm_ch.groupby(['ppid', 'AdmissionDate', 'ph_code']).size().reset_index(name=tn)
        READ_mind = cm_ch.groupby(['ppid', 'ph_code', 'AdmissionDate'])['EventDate'].min().reset_index(name='first_date')
        READ_maxd = cm_ch.groupby(['ppid', 'ph_code', 'AdmissionDate'])['EventDate'].max().reset_index(name='last_date')
        READ_mind[dfn] = ((READ_mind['AdmissionDate'] + timedelta(days=0)) - pd.to_datetime(READ_mind['first_date'])).dt.days
        READ_maxd[dln] = ((READ_maxd['AdmissionDate'] + timedelta(days=0)) - pd.to_datetime(READ_maxd['last_date'])).dt.days
        READ_ids = pd.merge(READ_ids, READ_mind[['ppid', 'AdmissionDate', 'ph_code', dfn]], how='left', on=['ppid', 'AdmissionDate', 'ph_code'])
        READ_ids = pd.merge(READ_ids, READ_maxd[['ppid', 'AdmissionDate', 'ph_code', dln]], how='left', on=['ppid', 'AdmissionDate', 'ph_code'])
        READ_piv = READ_ids.set_index(['ppid', 'AdmissionDate', 'ph_code']).unstack()
        READ_piv.columns = map(rename, READ_piv)
        READ_piv = READ_piv.reset_index()
        READ_total = READ_ids.groupby(['ppid', 'AdmissionDate'])['ph_code'].nunique().reset_index(name=gn)
        cur_coh = pd.merge(cur_coh, READ_sum, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, READ_piv, how='left', on=['ppid', 'AdmissionDate'])
        full_coh = pd.concat([full_coh, cur_coh], axis=0)

    return full_coh

In [ ]:
gp_sel = prep_pheno_features(gp_data, inp_ch)
inp_ch = get_pheno_features(gp_sel, inp_ch)

In [ ]:
inp_ch.columns.tolist()

In [ ]:
for col in inp_ch[list(col for col in inp_ch.columns if 'total_GP_cmr' in col)]:
    print(inp_ch.groupby('coh_idx')[col].value_counts())

In [ ]:
smr_sel = prep_pheno_features(smr_data, inp_ch, max_events=1, fn='', ctype='Code')
inp_ch = get_pheno_features(smr_sel, inp_ch, tn='total_SMR_cmr',
                      dfn='dsf_SMR_diag', dln='dsl_SMR_diag', gn='SMR_diag')

In [ ]:
for col in inp_ch[list(col for col in inp_ch.columns if 'dsl_SMR_diag' in col)]:
    print(inp_ch.groupby('coh_idx')[col].value_counts())

In [ ]:
inp_ch.shape

In [ ]:
### Combine GP and SMR summary data
read_cond = pd.read_csv('')
smr_cond = pd.read_csv('')
int_cond = list(set(read_cond['ltc_code']) & set(smr_cond['ltc_code']))
all_cond = list(set(read_cond['ltc_code']) | set(smr_cond['ltc_code']))
for cond in int_cond:
    inp_ch['dsf_'+cond] = inp_ch[['dsf_SMR_diag_' + cond, 'dsf_GP_diag_' + cond]].max(axis=1)
    inp_ch['dsl_'+cond] = inp_ch[['dsl_SMR_diag_' + cond, 'dsl_GP_diag_' + cond]].min(axis=1)

for cond in all_cond:
    if (('total_GP_cmr_' + cond) in inp_ch.columns) and (('total_SMR_cmr_' + cond) in inp_ch.columns):
        inp_ch['prev_diag_'+cond] = np.where((inp_ch['total_GP_cmr_'+cond]>0)|(inp_ch['total_SMR_cmr_'+cond]>0), 1, 0)
        inp_ch['num_diag_'+cond] = inp_ch[['total_GP_cmr_'+cond,'total_SMR_cmr_'+cond]].sum(axis=1).fillna(0).astype(np.int16)
    elif (('total_GP_cmr_' + cond) not in inp_ch.columns) and (('total_SMR_cmr_' + cond) in inp_ch.columns):
        inp_ch['prev_diag_'+cond] = np.where(inp_ch['total_SMR_cmr_'+cond]>0, 1, 0)
        inp_ch['num_diag_'+cond] = inp_ch['total_SMR_cmr_'+cond].fillna(0).astype(np.int16)
    elif (('total_GP_cmr_' + cond) in inp_ch.columns) and (('total_SMR_cmr_' + cond) not in inp_ch.columns):
        inp_ch['prev_diag_'+cond] = np.where(inp_ch['total_GP_cmr_'+cond]>0, 1, 0)
        inp_ch['num_diag_'+cond] = inp_ch['total_GP_cmr_'+cond].fillna(0).astype(np.int16)

inp_ch['total_unique_conditions'] = inp_ch[['prev_diag_' + cond for cond in all_cond]].sum(axis=1).astype(np.int16)
inp_ch['total_longterm_conditions'] = inp_ch[[col for col in inp_ch if 'prev_diag_physltc' in col or 'prev_diag_menltc' in col]].sum(axis=1).astype(np.int8)
inp_ch['total_physlongterm_conditions'] = inp_ch[[col for col in inp_ch if 'prev_diag_physltc' in col]].sum(axis=1).astype(np.int8)
inp_ch['total_menlongterm_conditions'] = inp_ch[[col for col in inp_ch if 'prev_diag_menltc' in col]].sum(axis=1).astype(np.int8)
inp_ch['phys_men_multimorbidity'] = np.where((inp_ch['total_physlongterm_conditions'] >= 1)&(inp_ch['total_menlongterm_conditions'] >= 1), 1, 0).astype(np.int8)
inp_ch = inp_ch.drop(inp_ch.filter(
    regex='dsf_SMR_diag|dsl_SMR_diag|total_SMR_cmr|dsf_GP_diag|dsl_GP_diag|total_GP_cmr|num_diag').columns, axis=1)

In [ ]:
inp_ch.columns.tolist()

In [ ]:
inp_ch.shape, inp_ch.ppid.nunique()

In [ ]:
inp_ch.phys_men_multimorbidity.value_counts()

In [ ]:
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)

#### SMR00 outpatient attendances

In [ ]:
smr00_data = pd.read_csv('', sep='\t', low_memory=False)

In [ ]:
def get_outp_features(outp_data, inp_ch, n_cohorts=10, window=15, max_att=1000):
    SMR_sel = outp_data[['ppid', 'CLINIC_DATE', 'SPECIALTY', 'CLINIC_ATTENDANCE']]
    SMR_sel = SMR_sel[SMR_sel.ppid.isin(inp_ch.ppid.unique())]
    SMR_sel['CLINIC_DATE'] = pd.to_datetime(SMR_sel['CLINIC_DATE'])
    SMR_sel = SMR_sel.dropna()
    smr_specs = SMR_sel.groupby('SPECIALTY').size().reset_index(name='num_specs')
    smr_specs = smr_specs['SPECIALTY'].loc[smr_specs['num_specs'] > max_att]
    
    ### Pipeline
    full_coh = pd.DataFrame()
    inp_ch['HOSP_adt'] = pd.to_datetime(inp_ch['HOSP_adt'])
    inp_ch['AdmissionDate'] = pd.to_datetime(inp_ch['AdmissionDate'])
    for i in range(1, n_cohorts+1, window):
        print('Processing cohort:', i)
        cur_coh = inp_ch[inp_ch['coh_idx']==i]
        smr_ch = pd.merge(SMR_sel, cur_coh[['ppid', 'AdmissionDate']], how='left', on='ppid')
        smr_ch = smr_ch[((smr_ch['AdmissionDate'] + timedelta(days=0)) > smr_ch['CLINIC_DATE'])&(smr_ch.ppid.isin(cur_coh.ppid.unique()))]
        SMR_sum = smr_ch.groupby(['ppid', 'AdmissionDate']).size().reset_index(name='num_outp_attendances')
        SMR_min = smr_ch.groupby(['ppid', 'AdmissionDate'])['CLINIC_DATE'].min().reset_index()
        SMR_max = smr_ch.groupby(['ppid', 'AdmissionDate'])['CLINIC_DATE'].max().reset_index()
        SMR_min['dsf_outp_att'] = ((SMR_min['AdmissionDate'] + timedelta(days=0)) - SMR_min['CLINIC_DATE']).dt.days
        SMR_max['dsl_outp_att'] = ((SMR_max['AdmissionDate'] + timedelta(days=0)) - SMR_max['CLINIC_DATE']).dt.days
        SMR_sum = pd.merge(SMR_sum, SMR_min[['ppid', 'AdmissionDate', 'dsf_outp_att']], how='left', on=['ppid', 'AdmissionDate'])
        SMR_sum = pd.merge(SMR_sum, SMR_max[['ppid', 'AdmissionDate', 'dsl_outp_att']], how='left', on=['ppid', 'AdmissionDate'])
        smr_spec_each = smr_ch[['ppid', 'AdmissionDate', 'SPECIALTY']]
        smr_spec_each['SPECIALTY'] = np.where(smr_spec_each['SPECIALTY'].isin(list(smr_specs)), smr_spec_each['SPECIALTY'], 'Other')
        smr_spec_each = smr_spec_each.groupby(['ppid', 'AdmissionDate', 'SPECIALTY']).size().reset_index(name='num_outp_att')
        smr_spec_each = smr_spec_each.pivot_table(index='ppid', columns='SPECIALTY', values='num_outp_att', aggfunc='sum', fill_value=0,
                                                  dropna=False)
        smr_spec_each.columns = ['num_outp_att_' + str(col).strip() for col in smr_spec_each.columns]
        smr_spec_each.reset_index(inplace=True)
        smr_spec_each = smr_spec_each.rename(columns={'index':'ppid'})
        smr_dna_spec = smr_ch[['ppid', 'AdmissionDate', 'SPECIALTY', 'CLINIC_ATTENDANCE']]
        smr_dna_spec['SPECIALTY'] = np.where(smr_dna_spec['SPECIALTY'].isin(list(smr_specs)), smr_dna_spec['SPECIALTY'], 'Other')
        smr_dna_spec = smr_dna_spec.loc[smr_dna_spec['CLINIC_ATTENDANCE'] == 8]
        smr_dna_spec = smr_dna_spec.groupby(['ppid', 'AdmissionDate', 'SPECIALTY']).size().reset_index(name='num_outp_att_dna')
        smr_dna_spec = smr_dna_spec.pivot_table(index='ppid', columns='SPECIALTY', values='num_outp_att_dna', aggfunc='sum', fill_value=0,
                                                  dropna=False)
        smr_dna_spec.columns = ['num_outp_att_dna_' + str(col).strip() for col in smr_dna_spec.columns]
        smr_dna_spec.reset_index(inplace=True)
        smr_dna_spec = smr_dna_spec.rename(columns={'index':'ppid'})
        cur_coh = pd.merge(cur_coh, SMR_sum, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, smr_spec_each, how='left', on=['ppid'])
        cur_coh = pd.merge(cur_coh, smr_dna_spec, how='left', on=['ppid'])
        full_coh = pd.concat([full_coh, cur_coh], axis=0)

    return full_coh

In [ ]:
inp_ch = get_outp_features(smr00_data, inp_ch)

In [ ]:
inp_ch.columns.tolist()

In [ ]:
for col in inp_ch[list(col for col in inp_ch.columns if 'num_outp' in col)]:
    print(inp_ch.groupby('coh_idx')[col].value_counts())

In [ ]:
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='REDACTED', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='Other', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='other', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='OTHER', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='_NA', axis=1).columns, axis=1)

In [ ]:
inp_ch.columns.tolist()

#### SMR01 inpatient history

In [ ]:
smr01_data = pd.read_csv('', sep='\t', low_memory=False)

In [ ]:
smr01_data.columns

In [ ]:
smr01_data['ADMISSION_TYPE'].value_counts()

In [ ]:
def get_admission_periods(smr_data):
    smr_data_tf = smr_data.sort_values(['ppid', 'ADMISSION_DATE', 'DISCHARGE_DATE'])
    smr_data_tf = smr_data_tf[['ppid', 'ADMISSION_DATE', 'DISCHARGE_DATE', 'SPECIALTY']]
    smr_data_tf['overlap'] = (smr_data_tf.groupby('ppid').apply(lambda x:(x.DISCHARGE_DATE.shift() - x.ADMISSION_DATE) >= timedelta(0))).reset_index(level=0, drop=True)
    idx_col = []
    ctr = 1
    for idx, r in smr_data_tf.iterrows():
        if not r.overlap and idx!=0:
            ctr+=1
        idx_col.append(ctr)
    smr_data_tf['idx'] = idx_col
    return smr_data_tf

def get_inp_features(inp_data, inp_ch, n_cohorts=10, window=15, max_att=1000):
    SMR_sel = inp_data[['ppid', 'ADMISSION_DATE', 'DISCHARGE_DATE', 'SPECIALTY', 'ADMISSION_TYPE']]
    ## include only unscheduled/urgent admission info
    SMR_sel = SMR_sel[SMR_sel['ADMISSION_TYPE']>=20].drop('ADMISSION_TYPE', axis=1)
    SMR_sel = SMR_sel[SMR_sel.ppid.isin(inp_ch.ppid.unique())]
    SMR_sel['ADMISSION_DATE'] = pd.to_datetime(SMR_sel['ADMISSION_DATE'])
    SMR_sel['DISCHARGE_DATE'] = pd.to_datetime(SMR_sel['DISCHARGE_DATE'])
    SMR_sel = SMR_sel.dropna()
    print(SMR_sel.shape)
    SMR_sel = get_admission_periods(SMR_sel)
    SMR_start = SMR_sel.groupby(['ppid', 'idx'])['ADMISSION_DATE'].min().reset_index(name='start_date')
    SMR_end = SMR_sel.groupby(['ppid', 'idx'])['DISCHARGE_DATE'].max().reset_index(name='end_date')
    SMR_sp = SMR_sel.groupby(['ppid', 'idx'])['SPECIALTY'].last().reset_index()
    SMR_full = pd.merge(SMR_start, SMR_end, how='left', on=['ppid', 'idx'])
    SMR_full = pd.merge(SMR_full, SMR_sp, how='left', on=['ppid', 'idx'])
    SMR_full['inp_hospital_days'] = (SMR_full.end_date - SMR_full.start_date).dt.days + 1
    SMR_full['stay>7d'] = np.where(SMR_full['inp_hospital_days'] > 7, 1, 0)
    SMR_full['stay>14d'] = np.where(SMR_full['inp_hospital_days'] > 14, 1, 0)
    SMR_full['stay>21d'] = np.where(SMR_full['inp_hospital_days'] > 21, 1, 0)
    smr_specs = SMR_full.groupby('SPECIALTY').size().reset_index(name='num_specs')
    smr_specs = smr_specs['SPECIALTY'].loc[smr_specs['num_specs'] > max_att]
    ### Pipeline
    full_coh = pd.DataFrame()
    inp_ch['HOSP_adt'] = pd.to_datetime(inp_ch['HOSP_adt'])
    inp_ch['AdmissionDate'] = pd.to_datetime(inp_ch['AdmissionDate'])
    for i in range(1, n_cohorts+1, window):
        print('Processing cohort:', i)
        cur_coh = inp_ch[inp_ch['coh_idx']==i]
        smr_ch = pd.merge(SMR_full, cur_coh[['ppid', 'AdmissionDate']], how='left', on='ppid')
        smr_ch = smr_ch[((smr_ch['AdmissionDate'] + timedelta(days=0)) > smr_ch['start_date'])&(smr_ch.ppid.isin(cur_coh.ppid.unique()))]
        SMR_sum = smr_ch.groupby(['ppid', 'AdmissionDate']).size().reset_index(name='num_inp_attendances')
        SMR_sum_days = smr_ch.groupby(['ppid', 'AdmissionDate'])['inp_hospital_days'].sum().reset_index(name='total_inp_hospital_days')
        SMR_sum_days_m = smr_ch.groupby(['ppid', 'AdmissionDate'])['inp_hospital_days'].median().reset_index(name='median_inp_hospital_days')
        SMR_sum_los1 = smr_ch.groupby(['ppid', 'AdmissionDate'])['stay>7d'].sum().reset_index(name='total_inp_hospital_days>7d')
        SMR_sum_los2 = smr_ch.groupby(['ppid', 'AdmissionDate'])['stay>14d'].sum().reset_index(name='total_inp_hospital_days>14d')
        SMR_sum_los3 = smr_ch.groupby(['ppid', 'AdmissionDate'])['stay>21d'].sum().reset_index(name='total_inp_hospital_days>21d')
        SMR_full_lyr = smr_ch[smr_ch.start_date >= ((smr_ch['AdmissionDate'] + timedelta(days=0)) - pd.DateOffset(days=365))]
        SMR_sum_lyr = SMR_full_lyr.groupby(['ppid', 'AdmissionDate']).size().reset_index(name='num_inp_attendances_lyr')
        SMR_sum_days_lyr = SMR_full_lyr.groupby(['ppid', 'AdmissionDate'])['inp_hospital_days'].sum().reset_index(name='total_inp_hospital_days_lyr')
        SMR_sum_days_m_lyr = SMR_full_lyr.groupby(['ppid', 'AdmissionDate'])['inp_hospital_days'].median().reset_index(name='median_inp_hospital_days_lyr')
        SMR_sum_los1_lyr = SMR_full_lyr.groupby(['ppid', 'AdmissionDate'])['stay>7d'].sum().reset_index(name='total_inp_hospital_days>7d_lyr')
        SMR_sum_los2_lyr = SMR_full_lyr.groupby(['ppid', 'AdmissionDate'])['stay>14d'].sum().reset_index(name='total_inp_hospital_days>14d_lyr')
        SMR_sum_los3_lyr = SMR_full_lyr.groupby(['ppid', 'AdmissionDate'])['stay>21d'].sum().reset_index(name='total_inp_hospital_days>21d_lyr')

        SMR_min = smr_ch.groupby(['ppid', 'AdmissionDate'])['start_date'].min().reset_index()
        SMR_max = smr_ch.groupby(['ppid', 'AdmissionDate'])['start_date'].max().reset_index()
        SMR_min['dsf_inp_att'] = ((SMR_min['AdmissionDate'] + timedelta(days=0)) - SMR_min['start_date']).dt.days
        SMR_max['dsl_inp_att'] = ((SMR_max['AdmissionDate'] + timedelta(days=0)) - SMR_max['start_date']).dt.days
        SMR_sum = pd.merge(SMR_sum, SMR_min[['ppid', 'AdmissionDate', 'dsf_inp_att']], how='left', on=['ppid', 'AdmissionDate'])
        SMR_sum = pd.merge(SMR_sum, SMR_max[['ppid', 'AdmissionDate', 'dsl_inp_att']], how='left', on=['ppid', 'AdmissionDate'])

        smr_spec_each = smr_ch[['ppid', 'AdmissionDate', 'SPECIALTY']]
        smr_spec_each['Specialty'] = np.where(smr_spec_each['SPECIALTY'].isin(list(smr_specs)), smr_spec_each['SPECIALTY'], 'Other')
        smr_spec_each = smr_spec_each.groupby(['ppid', 'AdmissionDate', 'SPECIALTY']).size().reset_index(name='num_inp_att')
        smr_spec_each = smr_spec_each.pivot_table(index='ppid', columns='SPECIALTY', values='num_inp_att', aggfunc='sum', fill_value=0,
                                                  dropna=False)
        smr_spec_each.columns = ['num_inp_att_' + str(col).strip() for col in smr_spec_each.columns]
        smr_spec_each.reset_index(inplace=True)
        smr_spec_each = smr_spec_each.rename(columns={'index':'ppid'})
        
        
        cur_coh = pd.merge(cur_coh, SMR_sum, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_days, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_days_m, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_los1, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_los2, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_los3, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_lyr, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_days_lyr, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_days_m_lyr, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_los1_lyr, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_los2_lyr, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, SMR_sum_los3_lyr, how='left', on=['ppid', 'AdmissionDate'])
        cur_coh = pd.merge(cur_coh, smr_spec_each, how='left', on=['ppid'])
    
        full_coh = pd.concat([full_coh, cur_coh], axis=0)

    return full_coh

In [ ]:
inp_ch = get_inp_features(smr01_data, inp_ch)

In [ ]:
for col in inp_ch[list(col for col in inp_ch.columns if 'num_inp_att' in col)]:
    print(inp_ch.groupby('coh_idx')[col].value_counts())

In [ ]:
inp_ch.columns.tolist()

In [ ]:
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='num_inp_att_A81', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='num_inp_att_H1', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='num_inp_att_CC', axis=1).columns, axis=1)
inp_ch = inp_ch.drop(columns=inp_ch.filter(like='inp_hospital_days', axis=1).columns, axis=1)

In [ ]:
inp_ch[[col for col in inp_ch.columns if col.endswith('_d')]]

#### Deal with NAs and export

In [ ]:
inp_ch[list(col for col in inp_ch.columns if 'num_inp' in col)] = inp_ch[list(col for col in inp_ch.columns if 'num_inp' in col)].fillna(0).astype(np.int16)
inp_ch[list(col for col in inp_ch.columns if 'num_outp' in col)] = inp_ch[list(col for col in inp_ch.columns if 'num_outp' in col)].fillna(0).astype(np.int16)
inp_ch[list(col for col in inp_ch.columns if 'dsf' in col)] = inp_ch[list(col for col in inp_ch.columns if 'dsf' in col)].fillna(39999).astype(np.int32)
inp_ch[list(col for col in inp_ch.columns if 'dsl' in col)] = inp_ch[list(col for col in inp_ch.columns if 'dsl' in col)].fillna(39999).astype(np.int32)
inp_ch[[col for col in inp_ch.columns if col.endswith('_d')]] = inp_ch[[col for col in inp_ch.columns if col.endswith('_d')]].fillna(39999).astype(np.int32)
inp_ch[[col for col in inp_ch.columns if col.endswith('_d')]] = np.where(inp_ch[[col for col in inp_ch.columns if col.endswith('_d')]]==-9999.0, 39999, inp_ch[[col for col in inp_ch.columns if col.endswith('_d')]])
inp_ch[list(col for col in inp_ch.columns if 'total_inp_' in col)] = inp_ch[list(col for col in inp_ch.columns if 'total_inp_' in col)].fillna(0).astype(np.int32)
inp_ch[list(col for col in inp_ch.columns if 'median_inp_' in col)] = inp_ch[list(col for col in inp_ch.columns if 'median_inp_' in col)].fillna(0.0).astype(np.float32)
inp_ch[list(col for col in inp_ch.columns if 'prev_diag_' in col)] = inp_ch[list(col for col in inp_ch.columns if 'prev_diag_' in col)].fillna(0).astype(np.int8)
inp_ch[list(col for col in inp_ch.columns if 'total_unique_conditions' in col)] = inp_ch[list(col for col in inp_ch.columns if 'total_unique_conditions' in col)].fillna(0).astype(np.int8)
inp_ch[list(col for col in inp_ch.columns if col.endswith('_rm'))] = inp_ch[list(col for col in inp_ch.columns if col.endswith('_rm'))].round(2).astype(np.float32)
inp_ch[list(col for col in inp_ch.columns if col.endswith('_rs'))] = inp_ch[list(col for col in inp_ch.columns if col.endswith('_rs'))].round(2).astype(np.float32)

In [ ]:
inp_ch.isnull().any().sum()

In [ ]:
inp_ch.columns.tolist()

In [ ]:
inp_ch.shape, inp_ch.ppid.nunique()

In [ ]:
inp_ch['AdmissionDate'] = inp_ch['AdmissionDate'].astype('category')
inp_ch['HOSP_adt'] = inp_ch['HOSP_adt'].astype('category')
inp_ch['DischargeDate'] = inp_ch['DischargeDate'].astype('category')
inp_ch['HOSP_ddt'] = inp_ch['HOSP_ddt'].astype('category')

In [ ]:
pd.DataFrame(inp_ch.dtypes).to_csv('')

In [ ]:
inp_ch.sort_values(['ppid', 'HOSP_adt']).to_csv('', index=False)